## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import keras
import numpy as np
from argparse import Namespace
from keras.datasets import cifar10
from keras.utils import to_categorical
from resnet_builder import resnet

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
training_size = len(x_train)
testing_size = len(y_train)
img_shape = x_train.shape[1:]
num_class = len(np.unique(y_train))
print('Training Size: {:d}'.format(training_size))
print('Test Size: {:d}'.format(training_size))
print('Image Size: {}'.format(img_shape))
print('Number of Class: {:d}'.format(num_class))

Training Size: 50000
Test Size: 50000
Image Size: (32, 32, 3)
Number of Class: 10


In [4]:
EPOCH = 5
BATCH_SIZE = 256
LEARNING_RATE = 1e-5
STEP_PER_EPOCH = np.ceil(training_size/BATCH_SIZE).astype(int)

In [5]:
def preprocess_x(x, flatten=False):
    x = x / 255.
    if flatten:
        x = x.reshape(len(x), -1)
    return x

x_train = preprocess_x(x_train)
x_test = preprocess_x(x_test)

y_train = to_categorical(y_train, num_class)
y_test = to_categorical(y_test, num_class)

In [6]:
def data_generator(x, y, batch_size):
    while True:
        for ind in range(0, len(x), batch_size):
            batch_x = x[ind: ind + batch_size, :]
            batch_y = y[ind: ind + batch_size, :]
            yield (batch_x, batch_y)

training_generator = data_generator(x_train, y_train, BATCH_SIZE)
testing_generator = data_generator(x_test, y_test, BATCH_SIZE)

In [7]:
resnet = resnet(input_shape=img_shape)

In [ ]:
resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

resnet.fit_generator(training_generator, steps_per_epoch=STEP_PER_EPOCH, epochs=EPOCH)

Epoch 1/5
  4/196 [..............................] - ETA: 23:48 - loss: 2.9628 - acc: 0.1338

In [ ]:
resnet.evaluate_generator(testing_generator, steps=STEP_PER_EPOCH)